In [1]:
import pandas as pd 
import numpy as np

In [2]:
df = pd.read_csv('../input/bankdataset/bank-additional-full.csv')
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,no


### Data Preprocessing

In [3]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
age,41188.0,40.024060,10.421250,17.000,32.000,38.000,47.000,98.000
duration,41188.0,258.285010,259.279249,0.000,102.000,180.000,319.000,4918.000
campaign,41188.0,2.567593,2.770014,1.000,1.000,2.000,3.000,56.000
pdays,41188.0,962.475454,186.910907,0.000,999.000,999.000,999.000,999.000
previous,41188.0,0.172963,0.494901,0.000,0.000,0.000,0.000,7.000
emp.var.rate,41188.0,0.081886,1.570960,-3.400,-1.800,1.100,1.400,1.400
cons.price.idx,41188.0,93.575664,0.578840,92.201,93.075,93.749,93.994,94.767
cons.conf.idx,41188.0,-40.502600,4.628198,-50.800,-42.700,-41.800,-36.400,-26.900
euribor3m,41188.0,3.621291,1.734447,0.634,1.344,4.857,4.961,5.045


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [6]:
# create dummies for categorical / string variables
df_dummies = pd.get_dummies(df, columns=['day_of_week',
                                        'job',
                                        'marital',
                                        'education',
                                        'default', 
                                        'housing',
                                        'loan',
                                        'contact',
                                        'month',
                                        'poutcome'],
                           drop_first=True)

In [7]:
# split dataset
from sklearn.model_selection import train_test_split

df2 = df_dummies.drop('y', axis = 1)
X = df2
y = df_dummies['y']

X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size = 0.30)


### Training and Testing Random Forest Classifier

In [8]:
# fitting a random forest classifier
from sklearn.ensemble import RandomForestClassifier

# specifying the random forest n estimator specifiecies the number of tres
rfc = RandomForestClassifier(n_estimators=1000)
rfc.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000)

### Additional : Accesing Confusion Matrix

In [11]:
# getting our predictions
y_pred = rfc.predict(X_test)

# printing the accuracy, confusion matrix, and the classification report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

y_actu = pd.Series(y_test, name='Actual')
df_confusion = pd.crosstab(y_actu, y_pred)
acc = accuracy_score(y_test, y_pred, normalize=True)
print(f'Model Accuracy Score {acc}')
print(df_confusion)
print(classification_report(y_test, y_pred))

Model Accuracy Score 0.9178603220846484
col_0      no  yes
Actual            
no      10677  362
yes       653  665
              precision    recall  f1-score   support

          no       0.94      0.97      0.95     11039
         yes       0.65      0.50      0.57      1318

    accuracy                           0.92     12357
   macro avg       0.79      0.74      0.76     12357
weighted avg       0.91      0.92      0.91     12357

